In [0]:
%sql
select 'count_of_appointments_details' as table_name,count(*) as record_count from appointments_raw
union all
select 'count_of_patients_details' as table_name ,count(*) as record_count from patients_raw
union all
select 'count_of_slots' as table_name,count(*) as record_count from slots_raw;


Clean/Transform Appointment Table. 
Convert string dates to proper DATE type,Remove duplicates,Keep only valid rows

In [0]:
%sql
CREATE OR REPLACE TABLE appointments AS
SELECT DISTINCT
    appointment_id AS AppointmentID,
    patient_id AS PatientID,
    TRY_CAST(scheduling_date AS DATE) AS ScheduledDate,
    TRY_CAST(appointment_date AS DATE) AS AppointmentDate,
    date_format(appointment_date, 'EEEE') AS AppointmentDay,
    status,
    CASE 
        WHEN Status IN ('attended', 'did not attend','cancelled') THEN Status
        ELSE 'Unknown'
    END AS CleanedStatus
FROM appointments_raw
where appointment_id is not null;

In [0]:
%sql
select * from appointments

Clean/Transform Patients Table
Remove invalid/null patient IDs, Standardize gender values and Drop duplicates

In [0]:
%sql
CREATE OR REPLACE TABLE patients AS
SELECT DISTINCT
    patient_id AS PatientID,
    upper(name) AS PatientName,
    CASE 
        WHEN LOWER(sex) IN ('male', 'm') THEN 'Male'
        WHEN LOWER(sex) IN ('female', 'f') THEN 'Female'
        ELSE 'Other'
    END AS Gender,
    dob as DateOfBirth,
    TIMESTAMPDIFF(YEAR, dob, CURDATE()) AS Age,
    CASE 
        WHEN LOWER(insurance) IS NOT NULL THEN 'Yes'
        ELSE 'No'
    END AS HasInsurance
FROM patients_raw
WHERE patient_id IS NOT NULL;

In [0]:
%sql
select * from patients where hasinsurance='No';

Clean/Transform Slots table
Check for null Slot ID, convert timestamp to only time format, user understandable availability

In [0]:
%sql
CREATE OR REPLACE TABLE slots AS
select DISTINCT
slot_id as slot_id,
appointment_date as appointment_date,
date_format(appointment_time,'HH:mm:ss') as appointment_time,
CASE when lower(is_available)='true' then 'Yes' 
when lower(is_available)='false' then 'No'
else 'unknown' end as is_available
from slots_raw
where slot_id is not null;

In [0]:
%sql
select * from slots;